In [ ]:
import polars as pl
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from cmocean import cm as cmo
import glidertools as gt

In [ ]:
dat = xr.open_dataset("C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/glider/pomBODCREQ-5915/unit_345/nc_files/L0-timeseries/Biocarbon_cabot.nc", chunks={"time": 1000})

In [ ]:
mask = (dat.depth < 400).compute()
dat_surf = dat.where(mask, drop=True)

In [ ]:
gt.plot(dat_surf['profile_index'], dat_surf['depth'], dat_surf['backscatter_700'], cmap=cmo.delta)
plt.title('b$_{bp}$ (m$^{-1}$)')
plt.show()

In [ ]:
bbp_1d = np.array(dat_surf['backscatter_700'])
prof_1d = np.array(dat_surf['profile_index'])
depth_1d = np.array(dat_surf['pressure'])

nan_mask = np.isnan(depth_1d)
valid_idx = np.where(~nan_mask)[0]  # Indices of non-NaN values
valid_values = depth_1d[valid_idx]   # Non-NaN values
depth_1d[nan_mask] = np.interp(np.where(nan_mask)[0], valid_idx, valid_values)


#bbp_horz = gt.cleaning.horizontal_diff_outliers(prof_1d, depth_1d, bbp_1d, depth_threshold=10, mask_frac=0.05)
bbp_baseline, bbp_spikes = gt.cleaning.despike(bbp_1d, 7, spike_method='minmax')


# fig, ax = plt.subplots(3, 1, figsize=[9, 6], sharex=True, dpi=90)

# gt.plot(prof_1d, depth_1d, bbp_1d, cmap=cmo.delta, ax=ax[0], robust=True)
# gt.plot(prof_1d, depth_1d, bbp_baseline, cmap=cmo.delta, ax=ax[1], robust=True)
# gt.plot(prof_1d, depth_1d, bbp_spikes, ax=ax[2], cmap=plt.cm.Spectral_r, vmin=0, vmax=0.004)

# [a.set_xlabel('') for a in ax]

# ax[0].set_title('Raw b$_{bp}$ (m$^{-1}$)')
# ax[1].set_title('Despiked b$_{bp}$ (m$^{-1}$)')
# ax[2].set_title('b$_{bp}$ (m$^{-1}$) spikes')

# plt.show()

In [ ]:
dat_surf['bbp700'] = ("time", bbp_baseline)

In [ ]:
# mask = (dat_surf.profile_index > 0).compute()
# prof_i = dat_surf.where(mask, drop = True)

# bbp_raw = prof_i['backscatter_700']
# bbp_despiked = prof_i['bbp700']
# depths = prof_i['depth'].compute()

# plt.figure(figsize=(8, 6))
# #plt.plot(bbp_raw, depths, label='Backscatter Raw', color='blue')
# plt.scatter(bbp_despiked, depths, label='BBP Despiked', color='orange')

# plt.gca().invert_yaxis()  # Depth increases downwards
# plt.xlabel("Backscatter / BBP")
# plt.ylabel("Depth (m)")
# plt.xlim(0,0.01)
# plt.legend()
# plt.title("Profile 352: Despiking result")
# plt.grid(True)
# plt.show()


In [ ]:
# gt.plot(dat_surf['profile_index'], dat_surf['depth'], dat_surf['chlorophyll'], cmap=cmo.delta)
# plt.title("Chlorophyll")
# plt.show()

In [ ]:
flr_iqr = gt.cleaning.outlier_bounds_iqr(dat_surf['chlorophyll'], multiplier=1000)

In [ ]:
flr_dark = gt.optics.fluorescence_dark_count(flr_iqr, dat_surf.depth)

In [ ]:
gt.plot(dat_surf['profile_index'], dat_surf['depth'], flr_dark, cmap=cmo.delta, robust=True)
plt.title('dark corrected fluorescence')
plt.show()

In [ ]:
flr_base, flr_spikes = gt.cleaning.despike(flr_dark, 11, spike_method='median')

# fig, ax = plt.subplots(2, 1, figsize=[9, 6], sharex=True, dpi=90)

# gt.plot(dat_surf['profile_index'], dat_surf['depth'], flr_base, cmap=cmo.delta, ax=ax[0], robust=True)
# gt.plot(dat_surf['profile_index'], dat_surf['depth'], flr_spikes, cmap=plt.cm.RdBu_r, ax=ax[1], vmin=0, vmax=4)

# [a.set_xlabel('') for a in ax]
# [a.set_ylim(300, 0) for a in ax]

# ax[0].set_title('Despiked Fluorescence')
# ax[1].set_title('Fluorescence spikes')

# plt.show()

In [ ]:
dat_surf = dat_surf.compute()

## Quenching correction

In the following cell we will perform quenching correction. 

The first step is to define night profiles, we selected every profiles between 10pm and 4am UTC, to be sure to avoid quenching. (Thus this can't be applied straight to other dataset where this time period might be quenched).
Then we define a fluo/bbp ratio on the first 40m of the profile, averaged.

On the day profiles, we identify the depth of the fluorescence maximum as the layer of potential quenching. We then compare the fluo/bbp ratio of the day profile to the night one. I it is lower that 95% of the night value (i.e. 5% difference) we recompute a corrected fluorescence as being bbp700 * fluo/bbp700_at_night

Everything is then appended back to the dataset. 

In [ ]:
#convert our fluo bbp dataset to a polar dataframe because it is easier.
data = [
    pl.Series("datetime",  dat_surf.time.data, dtype=pl.Datetime),
    pl.Series("prof", dat_surf.profile_index.data, dtype=pl.Float64),
    pl.Series("depth", dat_surf.depth.data, dtype=pl.Float32),
    pl.Series("bbp", dat_surf.bbp700.data, dtype=pl.Float32),
    pl.Series("fluo", flr_base.data, dtype=pl.Float32),
]

df = pl.DataFrame(data)

#define a function to detect if the profile is at day or night
def classify_day_night(dt):
    hour = dt.hour
    return "day" if 6 <= hour < 21 else "night"

# Add the new column
df = df.with_columns(
    pl.col("datetime").map_elements(lambda dt: classify_day_night(dt)).alias("day_night")
)

# Extract date part from datetime
df = df.with_columns(pl.col("datetime").dt.date().alias("date"))
df = df.with_columns(pl.col("depth").round(0).alias("depth_rounded"))
night_profiles = df.filter(
    (pl.col("day_night") == "night") & (pl.col("depth") <= 40)
)

# Compute the night fluo/bbp ratio
night_profiles = night_profiles.with_columns(
    (pl.col("fluo") / pl.col("bbp")).alias("fluo_bbp_ratio")
)

night_ratios_by_depth = night_profiles.group_by(["date"]).agg(
    pl.col("fluo_bbp_ratio").drop_nans().mean().alias("night_depth_ratio")
)

#compute the ratio for day profiles
day_profiles = df.filter(
    (pl.col("day_night") == "day") & (pl.col("depth") <= 40)
).with_columns(
    (pl.col("fluo") / pl.col("bbp")).alias("fluo_bbp_ratio")
)

#detect the quenched layer
quenched_layer = day_profiles.group_by("prof"
).agg(pl.col("fluo").drop_nans().max().alias("max_fluo_value")
)

max_depth_per_profile = (
    df.join(quenched_layer, on=["prof"])
    .filter(pl.col("fluo") == pl.col("max_fluo_value"))
    .select(["prof", "max_fluo_value", "depth"])
    .rename({"depth": "max_fluo_depth"})
).group_by(pl.col("prof")
).agg(pl.col("max_fluo_depth").drop_nans().mean())

# Join day profiles with corresponding night ratios by date and depth
day_profiles = day_profiles.join(
    night_ratios_by_depth, on=["date"], how="left"
).join(max_depth_per_profile, on = ["prof"], how = "left")
# Define a significant drop (e.g., 20% drop from night ratio)
day_profiles = day_profiles.with_columns(
    ((pl.col("fluo_bbp_ratio") < pl.col("night_depth_ratio") * 0.95) & (pl.col("depth") < pl.col("max_fluo_depth")))
    .alias("quenching_detected")
)
day_profiles = day_profiles.with_columns(
    pl.when(pl.col("quenching_detected"))
    .then(pl.col("bbp") * pl.col("night_depth_ratio"))
    .otherwise(pl.col("fluo"))
    .alias("fluo_unquenched")
)
df = df.join(day_profiles.select(["datetime", "depth", "fluo_unquenched"]), 
              on=["datetime", "depth"], how="left")

# Fill night profiles or uncorrected points with original fluo values
df = df.with_columns(
    pl.when(pl.col("fluo_unquenched").is_null())
    .then(pl.col("fluo"))
    .otherwise(pl.col("fluo_unquenched"))
    .alias("fluo_unquenched")
)

In [ ]:
dat_surf['fluo_corrected'] = ("time", df["fluo_unquenched"])
dat_surf['fluo'] = ("time", df["fluo"])

In [ ]:
# flr_qc, quench_layer = gt.optics.quenching_correction(
#     flr_base, dat_surf.bbp700, dat_surf.profile_index, dat_surf.depth, dat_surf.time.data.astype('datetime64[ms]'), dat_surf.latitude, dat_surf.longitude,
#     sunrise_sunset_offset=1, night_day_group = True, max_photic_depth = 50)

# fig, ax = plt.subplots(2, 1, figsize=[9, 6], sharex=True, dpi=90)

# gt.plot(dat_surf['profile_index'], dat_surf['depth'], flr_qc, cmap=cmo.delta, ax=ax[0], robust=True)
# gt.plot(dat_surf['profile_index'], dat_surf['depth'], quench_layer, cmap=plt.cm.RdBu_r, ax=ax[1], vmin=-.5, vmax=2)

# [a.set_xlabel('') for a in ax]
# [a.set_ylim(100, 0) for a in ax]

# ax[0].set_title('Quenching Corrected Fluorescence')
# ax[1].set_title('Quenching Layer')

# plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=[9, 6], sharex=True, dpi=90)

gt.plot(dat_surf['profile_index'], dat_surf['depth'], dat_surf['fluo'], cmap=cmo.delta, ax=ax[0], robust=True)
gt.plot(dat_surf['profile_index'], dat_surf['depth'], dat_surf['fluo_corrected'], cmap=cmo.delta, ax=ax[1], robust=True)
gt.plot(dat_surf['profile_index'], dat_surf['depth'], dat_surf['bbp700'], cmap=cmo.delta, ax=ax[2])

[a.set_xlabel('') for a in ax]
[a.set_ylim(100, 0) for a in ax]
#[a.set_xlim(400, 450) for a in ax]

ax[0].set_title("Raw Fluorescence")
ax[1].set_title('Quenching Corrected Fluorescence')
ax[2].set_title('bbp')

plt.show()

In [ ]:
dat_surf['fluo_dark'] = ("time", flr_dark.data)
#dat_surf['fluo_unquenched'] = ("time", flr_qc.data)

In [ ]:
mask = (dat_surf.profile_index == 440).compute()
prof_i = dat_surf.where(mask, drop = True)

flr_raw = prof_i['fluo']
flr_unquenched = prof_i['fluo_corrected']
depths = prof_i['depth'].compute()

plt.figure(figsize=(8, 6))
plt.plot(flr_raw, depths, label='Fluorescence Raw', color='blue')
plt.plot(flr_unquenched, depths, label='Fluorescence Unquenched', color='orange', linestyle='--')

plt.gca().invert_yaxis()  # Depth increases downwards
plt.xlabel("Fluorescence")
plt.ylabel("Depth (m)")
plt.legend()
plt.title("Profile: Fluorescence at Night")
plt.grid(True)
plt.show()


In [ ]:
mask = (dat_surf.profile_index == 372).compute()
prof_i = dat_surf.where(mask, drop = True)

flr_raw = prof_i['fluo']
flr_unquenched = prof_i['fluo_corrected']
depths = prof_i['depth'].compute()

plt.figure(figsize=(8, 6))
plt.plot(flr_raw, depths, label='Fluorescence Raw', color='blue')
plt.plot(flr_unquenched, depths, label='Fluorescence Unquenched', color='orange', linestyle='--')

plt.gca().invert_yaxis()  # Depth increases downwards
plt.xlabel("Fluorescence")
plt.ylabel("Depth (m)")
plt.legend()
plt.title("Profile: Fluorescence at day")
plt.grid(True)
plt.show()

Satellite PAR matchup

In [ ]:
import os

# Directory where your NetCDF files are stored (daily files)
ncdf_dir = "C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/satellite/par_mapped/"

# List all NetCDF files for the year (one file per day)
ncdf_files = sorted([f for f in os.listdir(ncdf_dir) if f.endswith('.nc')])

# Load the PAR data for each file and store it
daily_par_data = {}
for file in ncdf_files:
    # Open the dataset for the specific day
    ds = xr.open_dataset(os.path.join(ncdf_dir, file))
    
    # Extract PAR data (assuming it's named 'par', adjust if necessary)
    # Add a date key to use as the dictionary key for each day
    date = file.split('.')[1] # Extract the date from the filename (adjust based on filename format)
    
    # Store the data in a dictionary, with the date as the key
    daily_par_data[date] = ds["par"]  # Replace 'par' with the actual variable name in your files

# Example: inspect one of the datasets
print(daily_par_data["20240101"])  # Inspect the PAR data for January 1st, 2024

In [ ]:
from scipy.spatial import cKDTree
import datetime

# Function to find the nearest PAR value for each observation
def get_nearest_par(lat, lon, date_str, daily_par_data):
    # Check if the date is in the daily PAR data
    if date_str in daily_par_data and daily_par_data[date_str].size > 0:
        # Get PAR data for that date and find the nearest point
        par_data = daily_par_data[date_str]
        nearest_par = par_data.sel(lon=lon, lat=lat, method='nearest').values.flatten()[0]
    else:
        nearest_par = 0  # Default if no data for that date
    return nearest_par

In [ ]:
lon = dat_surf['longitude'].data
lat = dat_surf['latitude'].data
time =  [datetime.datetime.utcfromtimestamp(t.astype('datetime64[s]').astype(int)).strftime("%Y%m%d") for t in dat_surf['time'].data]
prof_index = dat_surf['profile_index'].data

df = pd.DataFrame({'lon' : lon, 'lat' : lat, 'time' : time, 'profile_index' : prof_index})
profile_means = (
    df.groupby("profile_index")[["lat", "lon"]]
    .mean()
    .reset_index()
)

In [ ]:
dates_series = df.groupby(['profile_index','time']).size().reset_index().rename(columns={0:'count'})


In [ ]:
df_to_match = pd.merge(profile_means, dates_series, how='left',on=['profile_index'])

In [ ]:
df_to_match.head()

In [ ]:
from tqdm import tqdm

# Step 3: Match PAR for each profile (with progress bar)
par_per_profile = []
for _, row in tqdm(df_to_match.iterrows(), total=len(df_to_match)):
    nearest_par = get_nearest_par(row["lat"], row["lon"], row["time"], daily_par_data)
    par_per_profile.append(nearest_par)

# Add PAR to the profile_means DataFrame
df_to_match["PAR_matched"] = par_per_profile

# Step 4: Merge PAR values back into the original DataFrame
df = df.merge(df_to_match[["profile_index", "PAR_matched"]], on="profile_index")

df.drop_duplicates(subset=['profile_index'], inplace = True)


In [ ]:
df.head()

In [ ]:
df_xr = df.to_xarray

In [ ]:
# Ensure DataFrame has the profile_index and PAR columns
df = df[["profile_index", "PAR_matched"]]

# Step 1: Merge DataFrame with xarray dataset's profile_index as a DataFrame
dat_surf_df = dat_surf.to_dataframe().reset_index()

# Step 2: Perform a left join on profile_index
merged_df = dat_surf_df.merge(df, on="profile_index", how="left")

# Step 3: Convert the merged DataFrame back into xarray
dat_surf = merged_df.set_index(["time"]).to_xarray()

# Step 4: Ensure the new PAR column is added properly
dat_surf['PAR'] = dat_surf['PAR_matched']
dat_surf = dat_surf.drop_vars('PAR_matched')  # Optional: drop the extra 'par' column


In [ ]:
# Step 1: Keep only rows with depth between 0 and 201 meters
filtered_df = merged_df[(merged_df['depth'] >= 0) & (merged_df['depth'] <= 201)].copy()

# Step 2: Create 1-meter bins (0 to 201 inclusive)
filtered_df['depth_bin'] = np.floor(filtered_df['depth']).astype(int)

# Step 3: Group by profile and depth bin, then average values
averaged_df = (
    filtered_df
    .groupby(['profile_index', 'depth_bin'])
    .mean()
    .reset_index()
)

# Step 4: Sort for cleaner output
averaged_df = averaged_df.sort_values(['profile_index', 'depth_bin']).reset_index(drop=True)


In [ ]:
averaged_df['BBP470'] = averaged_df['bbp700']/(470/400) 
averaged_df['carbon'] = 12128 * averaged_df['BBP470'] + 0.59

In [ ]:
averaged_df['time'] = pd.to_datetime(averaged_df['time'])

# Extract year, month, and day into new columns
averaged_df['year'] = averaged_df['time'].dt.year
averaged_df['month'] = averaged_df['time'].dt.month
averaged_df['day'] = averaged_df['time'].dt.day

In [ ]:
# Define the complete range of depth bins
depth_bins_full = list(range(201))

# Get the unique profile indices to ensure we process each profile separately
profile_indices = averaged_df['profile_index'].unique()

# Process each profile individually
new_rows = []
for profile in profile_indices:
    profile_df = averaged_df[averaged_df['profile_index'] == profile]
    
    # Only proceed if the profile has more than 180 rows
    if len(profile_df) > 180:
        existing_bins = set(profile_df['depth_bin'])
        missing_bins = [bin for bin in depth_bins_full if bin not in existing_bins]
        
        if missing_bins:
            # Take the first row of the existing profile to replicate
            first_row = profile_df.iloc[0].copy()
            
            for missing_bin in missing_bins:
                new_row = first_row.copy()
                new_row['depth_bin'] = missing_bin
                new_rows.append(new_row)

# Add the new rows to the original DataFrame if any rows were created
if new_rows:
    averaged_df = pd.concat([averaged_df, pd.DataFrame(new_rows)], ignore_index=True)

# Sort the DataFrame to keep depth_bin order neat
averaged_df = averaged_df.sort_values(by=['profile_index', 'depth_bin']).reset_index(drop=True)


In [ ]:
averaged_df[averaged_df['profile_index'] == 282].head(10)

In [ ]:
# Define time constants
day_start = 6  # Dawn (6 AM)
day_end = 20  # Dusk (6 PM)
t_noon = 13  # Noon time
day_length = day_end - day_start

# Extract hour of day as a decimal (e.g., 8:30 → 8.5)
averaged_df["hour_of_day"] = averaged_df["time"].dt.hour + averaged_df["time"].dt.minute / 60

# Define the weighted PAR function
def compute_weighted_par(hour, par_noon):
    """Compute weighted PAR based on time of day."""
    if hour  < day_start or hour > day_end:
        weight = 0
    else:
        weight = max(0, np.cos((np.pi * (hour - t_noon)) / (day_length / 2)))
    return par_noon * weight

# Apply the function row-wise to compute weighted PAR values
averaged_df["PAR_weighted"] = averaged_df.apply(
    lambda row: compute_weighted_par(row["hour_of_day"], row["PAR_matched"]), axis=1
)


In [ ]:
def running_mean(array, window_size=5):
    kernel = np.ones(window_size) / window_size
    smoothed_array = np.convolve(array, kernel, mode='same')  # 'same' ensures the output matches input size
    return smoothed_array

In [ ]:
from cbpm_argo import cbpm_argo
import numpy as np
from scipy.interpolate import interp1d
dfs = []
depth_grid = np.arange(0,200)

# Iterate through each unique profile index
for i in averaged_df['profile_index'].unique():
    # Filter for rows corresponding to the current 'JULD'
    temp_df = averaged_df[averaged_df['profile_index'] == i].iloc[0:200,].copy()  # Use `.copy()` to avoid warnings

    # Extract the pressure and chlorophyll values for interpolation
    pres_values = temp_df['depth_bin'].to_numpy()
    chl_values = temp_df['fluo_corrected'].to_numpy()
    carbon_values = temp_df['carbon'].to_numpy()

    # Apply the running mean smoothing
    chl_smoothed = running_mean(chl_values, window_size=5)
    carbon_smoothed = running_mean(carbon_values, window_size=5)

    # Check for valid data before interpolation (avoid NaN values)
    mask = ~np.isnan(pres_values) & ~np.isnan(chl_values)
    pres_values = pres_values[mask]
    chl_values = chl_values[mask]
    carbon_values = carbon_values[mask]

    # Interpolate the CHLA_ADJUSTED onto the depth grid (0 to 199)
    if len(pres_values) > 1:  # Ensure there's enough data to interpolate
        interpolator = interp1d(pres_values, chl_values, bounds_error=False, fill_value=np.nan)
        interpolated_chl = interpolator(depth_grid)
        
        interpolator = interp1d(pres_values, carbon_values, bounds_error=False, fill_value=np.nan)
        interpolated_carbon = interpolator(depth_grid)
        
    else:
        # If only one point or no valid data, fill with NaN
        print(temp_df['profile_index'].unique())
        interpolated_chl = np.full(depth_grid.shape, np.nan)
        interpolated_carbon = np.full(depth_grid.shape, np.nan)

    # Now we can extract other values and apply the cbpm_argo function
    chl_z = interpolated_chl
    Cphyto_z = interpolated_carbon
    irr = temp_df['PAR_weighted'].mean()  # Mean irradiance value
    year = int(temp_df['year'].mean())
    month = int(temp_df['month'].mean())
    day = int(temp_df['day'].mean())
    lat = temp_df['latitude'].mean()

    # Call the cbpm_argo function with interpolated data
    [pp_z, mu_z, par_z, prcnt_z, nutTempFunc_z, IgFunc_z, mzeu] = cbpm_argo(chl_z, Cphyto_z, irr, year, month, day, lat)

    size_max = len(temp_df)

    # Use .loc to explicitly assign new columns (expand results back into DataFrame)
    temp_df.loc[:, 'pp'] = pp_z[0:size_max]
    temp_df.loc[:, 'mu'] = mu_z[0:size_max]
    temp_df.loc[:, 'prcnt'] = prcnt_z[0:size_max]
    temp_df.loc[:, 'nutTempFunc'] = nutTempFunc_z[0:size_max]
    temp_df.loc[:, 'IgFunc'] = IgFunc_z[0:size_max]
    temp_df.loc[:, 'zeu'] = np.full(size_max, mzeu)

    # Append modified DataFrame to the list
    dfs.append(temp_df)


# Combine all DataFrames
final_df = pd.concat(dfs)

In [ ]:
dat_final = final_df.set_index(["time"]).to_xarray()

In [ ]:
len(final_df[final_df['profile_index'] == 282])

In [ ]:
final_df.iloc[0:201]

In [ ]:
dat_final

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=[9, 6], sharex=True, dpi=90)

gt.plot(dat_final['profile_index'], dat_final['depth'], dat_final['fluo_corrected'], cmap=cmo.delta, ax=ax[0], robust=True)
gt.plot(dat_final['profile_index'], dat_final['depth'], dat_final['bbp700'], cmap=cmo.matter, ax=ax[1], robust=True)
gt.plot(dat_final['profile_index'], dat_final['depth'], dat_final['pp'], cmap=plt.cm.RdBu_r, ax=ax[2])

[a.set_xlabel('') for a in ax]
[a.set_ylim(100, 0) for a in ax]
#[a.set_xlim(265, 275) for a in ax]

ax[0].set_title("Fluorescence")
ax[1].set_title('Carbon phyto')
ax[2].set_title('NPP (CbPM)')

plt.show()

In [ ]:
#dat_final = dat_final.drop_vars(['PAR_matched_x', 'PAR_matched_y'])
display(dat_final)
dat_final.to_netcdf('C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/glider/npp_ncdf/cabot_npp.nc')